In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?

In [2]:
import geopandas as gpd
import pandas as pd
from siuba import *
import numpy as np

from segment_speed_utils import helpers, gtfs_schedule_wrangling
from shared_utils import rt_dates, gtfs_utils_v2
import folium
import itertools

In [3]:
from update_vars import (analysis_date, AM_PEAK, PM_PEAK, EXPORT_PATH, GCS_FILE_PATH, PROJECT_CRS,
SEGMENT_BUFFER_METERS, AM_PEAK, PM_PEAK, HQ_TRANSIT_THRESHOLD, MS_TRANSIT_THRESHOLD)

# Export hq corridor linestring segments for walkshed analysis

See https://github.com/cal-itp/data-analyses/issues/1445

In [5]:
gdf = gpd.read_parquet(f'{GCS_FILE_PATH}all_bus.parquet')

In [7]:
hqtc = gdf.query('hq_transit_corr')

In [13]:
hqtc.columns

Index(['schedule_gtfs_dataset_key', 'route_id', 'route_key',
       'segment_sequence', 'geometry', 'hqta_segment_id', 'segment_direction',
       'route_direction', 'stop_id', 'am_max_trips', 'route_dir',
       'pm_max_trips', 'am_max_trips_hr', 'pm_max_trips_hr', 'route_dir_count',
       'hq_transit_corr', 'ms_precursor'],
      dtype='object')

In [9]:
# hqtc.explore() #  these are buffered...

In [10]:
segs = gpd.read_parquet(f'{GCS_FILE_PATH}hqta_segments.parquet') #  these are linestrings, but without frequency info

In [14]:
segs.columns

Index(['schedule_gtfs_dataset_key', 'route_id', 'route_key',
       'segment_sequence', 'geometry', 'hqta_segment_id', 'segment_direction',
       'route_direction'],
      dtype='object')

In [18]:
segs = segs[['hqta_segment_id', 'geometry']]

In [27]:
hqtc = hqtc.drop(columns=['geometry'])

In [29]:
hqtc_lines = pd.merge(segs, hqtc, on='hqta_segment_id')

In [31]:
# hqtc_lines.explore() #   this is the desired output

In [32]:
hqtc_lines.to_file(f'hq_corridor_linestrings_{analysis_date}.geojson')